In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade pip
# Installs the wheel compatible with CUDA 11 and cuDNN 8.2 or newer.
# Note: wheels only available on linux.
!pip install --upgrade "jax[cuda]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

In [ ]:
!pip install flax -U
!pip install tqdm

## Jax setup for colab

In [ ]:
# https://github.com/google/flax/issues/2263#issuecomment-1173424293
import sys
if 'google.colab' in sys.modules:
  import jax.tools.colab_tpu
  jax.tools.colab_tpu.setup_tpu()

## Dataset download
https://machinelearningmastery.com/develop-a-deep-learning-caption-generation-model-in-python/

In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip

In [ ]:
!wget https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip

In [ ]:
!unzip Flickr8k_Dataset.zip

In [ ]:
!unzip Flickr8k_text.zip

## Setup

In [ ]:
import torch
import numpy as np

In [ ]:
import tensorflow_hub as hub

In [ ]:
sentence_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
message = ["I am a sentence for which I would like to get its embedding."]

In [ ]:
embed = sentence_encoder(message)

In [ ]:
embed_np = np.array(embed)

In [ ]:
print(f'embed: {", ".join(map(str, list(np.squeeze(embed_np)[:3])))}, ...')

## DataLoader

In [ ]:
class CLIPDataset(torch.data.utils.Datasets):
  def __init__(self, image_files, captions):
    self.image_files = image_files
    self.captions = captions

  def __getitem__(self, index):
    pass

  def __len__(self):
    return len(self.image_files)